In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
from scipy.stats import norm
import time as ttt
import iisignature as iisig
from tqdm import *
from einops import rearrange
import signatory

In [2]:
import torch
import torch.nn as nn
import pandas as pd 

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

In [3]:
data_type=torch.float32

In [4]:
x0 = 100.0 # initial condition
sigma = 0.15 # volatility
d=20
K=100.0

segs=20
r = 0.05 # risk free rate
batch_size = 100 # batch size
steps=1000
T = 1 # maturity
dt = T/steps # mesh size

sqrt_dt=np.sqrt(dt); 
dt_new = T/segs # new mesh after shrinkage
level = 3 # truncation level logsig 91 sig 258

In [5]:
batch_in=100
MOMENTUM = 0.99
EPSILON = 1e-6
import warnings
warnings.filterwarnings("ignore")

In [6]:
def create_stock(x0,r,sigma,T,steps,n_path,dW):  
    s_vec=[];
    w0=np.ones((n_path,1,d))*1e-6
    dW=np.concatenate((w0,dW),axis=1) ## the first time slot is then 0
    s_vec.append(np.ones((n_path,d))*100.0)
    for i in range(steps): 
        s_vectemp=s_vec[-1]+ r*s_vec[-1]*dt+ dW[:,i+1,:]*s_vec[-1]*sigma
        s_vec.append(s_vectemp)
    BM_path=np.cumsum(dW,axis=1) ## find the cumulative sum
    S_path=rearrange(np.array(s_vec), 'b c h -> c b h') 
    return BM_path, S_path

def jointime(T,path): 
    n_path, steps, d=path.shape
    dt=T/(steps-1); 
    
    times=np.arange(0,T,dt) ## This can be taken out
    times=np.append(times,T); 
    times_vec=np.tile(times,[1,1]); 
    times_vec=np.transpose(times_vec)
    times_vec=np.tile(times_vec,[n_path,1,1])
    times_vec=np.concatenate((times_vec, path),axis=2)
    return times_vec


##  the dimension
$$\frac{d^{N+1}-1}{d-1}$$

In [7]:
embedding=signatory.Augment(in_channels=d+1,layer_sizes=(64,64,5),kernel_size=1,include_original=False,include_time=True)  

In [8]:
def ComputeMultiLevelSig(s_embed, number_of_segment,level):
    n_batch, nsteps, chanels = s_embed.shape
    s_pathclass = signatory.Path(s_embed,level) ## Create a path class 

    t_vec = np.arange(0, nsteps-1, int(nsteps / 20))
    t_vec = np.append(t_vec, nsteps-1)
    MultiLevelSig = []

    ll=torch.zeros(n_batch, 258) #iisig.sig(np.expand_dims(path[:,0,:],axis=1),depth)
    MultiLevelSig.append(ll)
    for i in range(len(t_vec)-1):    
    ## Notice that we only use the signature of the concatenation of time and space.
        MultiLevelSig.append(s_pathclass.signature(0, t_vec[i+1]+1)) ##if not
        #MultiLevelSig.append(path_class.signature(t_vec[i],t_vec[i+1]+1))
    MultiLevelSig=torch.stack(MultiLevelSig)  
    MultiLevelSig=rearrange(MultiLevelSig, 'b c h -> c b h') 
    return MultiLevelSig

In [9]:
### The x_ten here has to be a path
### The level of discretization is on the fine mesh
def terminal_loss(x_ten):
    res=torch.relu(torch.sum(x_ten,dim=(1,2))*dt/d-K)
    return res

In [10]:
def generate_samples(model,batch_in=100):
    """
    Produce the signature, dW for computation, YT the terminal condition
    x_ten, selection the path and location to be used
    """
    dW=np.sqrt(dt)*np.random.normal(size=(batch_in, steps,d))
    
    pth2=create_stock(x0,r,sigma,T,steps,batch_in,dW)
    BM_timePath=jointime(T,pth2[0]); 
    S_timePath=jointime(T,pth2[1]);
    
    ########### embed the long sequence to a lower dimension
    BM_timePath=torch.tensor(BM_timePath,dtype=data_type)
    S_timePath=torch.tensor(S_timePath,dtype=data_type)
    s_embed=model.embedding(S_timePath)
    ###########
    
    
    sigs=ComputeMultiLevelSig(s_embed, 20, 3)
    selection = np.linspace(0,steps, segs+1, dtype = np.int)

    BM_seg=BM_timePath[:,selection,1:]
    dW=BM_seg[:,1:,:]-BM_seg[:,:-1,:]

    dW=torch.tensor(dW,dtype=torch.float32)
    sigs=torch.tensor(sigs,dtype=torch.float32)
    S_timePath=torch.tensor(S_timePath,dtype=torch.float32)
    
    x_ten=S_timePath[:,:,1:]
    
    YT=torch.relu(torch.sum(x_ten,dim=(1,2))*dt/d-K)
    
    return sigs, dW, YT, x_ten, selection[:-1]

In [11]:
class Config(object):
    n_layer = 4
    batch_size = 1024
    valid_size = 1024
    
    
   ## We embed the squence dim=100 into a sequence of dim 5, then plus the time dimension we get 6. 
    dim=258; 
    Ntime=20; 
    delta=1/Ntime
    sqrt_deltaT=np.sqrt(1.0/Ntime); 
    lam=1; 

    logging_frequency = 100
    verbose = True
    y_init_range = [0, 1]
    
    num_hiddens = [dim,64,64,d] ## 256 ,256
    
def get_config(name):
    try:
        return globals()[name]
    except KeyError:
        raise KeyError("config not defined.")

cfg=get_config('Config')

In [12]:
class Dense(nn.Module): 
    def __init__(self,cin, cout, batch_norm=False, activate=True): 
        super(Dense,self).__init__()
        self.cin=cin; 
        self.cout=cout; 
        self.activate=activate; 
        
        self.linear=nn.Linear(self.cin,self.cout) #The linear layer
        #BatchNorm1d: it requires the input to be a correct size
        if batch_norm: 
            self.bn=nn.BatchNorm1d(cout,eps=EPSILON,momentum=MOMENTUM)
        else: 
            self.bn=None
      #  nn.init.normal_(self.linear.weight,std=5.0/np.sqrt(cin+cout))
        # This is the He initialization
        
    def forward(self,x): 
        x=self.linear(x)
        if self.bn is not None:
            x=self.bn(x)
        if self.activate:
            x=torch.tanh(x)
        return x 
    
class FFN(nn.Module):
    def __init__(self, config):
        super(FFN,self).__init__()
        self.config=config
        
        self.bn=nn.BatchNorm1d(config.num_hiddens[0],eps=EPSILON,momentum=MOMENTUM) ## So there is batch norm no problem
        # range(1,5): 1,2,3,4
        self.layers=[Dense(config.num_hiddens[i-1],config.num_hiddens[i]) for i in range(1, len(config.num_hiddens)-1)]
        self.layers+=[Dense(config.num_hiddens[-2], config.num_hiddens[-1],activate=False)]
        self.layers=nn.Sequential(*self.layers)
    
    def forward(self,x):
      #  x=self.bn(x)
        x=self.layers(x)
        return x 
    
class Lookback_PPDE_Backward(nn.Module):
    def __init__(self,cfg): 
        super(Lookback_PPDE_Backward,self).__init__()
        self.cfg=cfg
        self.Ntime=self.cfg.Ntime 
        self.mList=nn.ModuleList([FFN(self.cfg) for _ in range(self.Ntime)])
        self.embedding=signatory.Augment(in_channels=d+1,layer_sizes=(64,64,5),kernel_size=1,include_original=False,include_time=True)  
        
    def forward(self,batch_sig,batch_dW,batch_YT, batch_x, batch_sel): 
        Y=batch_YT
        for i in np.arange(segs-1,-1, -1):
            Y=Y-Y*r*T/segs-sigma*torch.sum( self.mList[i](batch_sig[:,i,:])*batch_dW[:,i,:],axis=1,keepdim=True)
            gi=terminal_loss(batch_x[:,:batch_sel[i],:])
            Y=torch.relu(gi-Y)+Y
        return Y

In [13]:
def loss_var(x):
    temp=torch.var(x)
    return temp

In [14]:
import torch.optim as optim
from torch.nn import Parameter
import math
model_PPDE_bw=Lookback_PPDE_Backward(cfg)
optimizer=optim.Adam(model_PPDE_bw.parameters(),lr=5e-4)
grad_clip=0.1
y0_mean=[];
loss_vec=[];
#scheduler=torch.optim.lr_scheduler.StepLR(optimizer, step_size=80, gamma=0.1)

In [15]:
#ss=generate_samples(model_PPDE_bw)

# dim 20

In [15]:
for i in range(120):
    batch_sig, batch_dw, batch_y,batch_x, batch_sel =generate_samples(model_PPDE_bw,batch_in=500)

    x_temp=model_PPDE_bw(batch_sig,batch_dw,batch_y,batch_x, batch_sel)
    loss_temp=loss_var(x_temp)

    optimizer.zero_grad()
    loss_temp.backward()
    optimizer.step()
#   scheduler.step()
    y0_val=x_temp.mean().cpu().detach().numpy()
    loss_val=loss_temp.cpu().detach().numpy()
    
    y0_mean.append(y0_val)
    loss_vec.append(loss_val)
    if grad_clip: 
        nn.utils.clip_grad_value_(model_PPDE_bw.parameters(), grad_clip)
    
    print("Iter:", i, 'The mean Y0 is', y0_val , 'Variance is:' ,loss_val)

Iter: 0 The mean Y0 is 2.6422036 Variance is: 3.0427287
Iter: 1 The mean Y0 is 2.7333388 Variance is: 2.8905716
Iter: 2 The mean Y0 is 2.6566775 Variance is: 3.115739
Iter: 3 The mean Y0 is 2.6990643 Variance is: 2.8678896
Iter: 4 The mean Y0 is 2.673203 Variance is: 3.337086
Iter: 5 The mean Y0 is 2.5073683 Variance is: 3.0237088
Iter: 6 The mean Y0 is 2.5579526 Variance is: 2.7055352
Iter: 7 The mean Y0 is 2.649607 Variance is: 2.773813
Iter: 8 The mean Y0 is 2.805558 Variance is: 3.32476
Iter: 9 The mean Y0 is 2.533758 Variance is: 2.9834495
Iter: 10 The mean Y0 is 2.466094 Variance is: 3.108196
Iter: 11 The mean Y0 is 2.5803285 Variance is: 2.9670677
Iter: 12 The mean Y0 is 2.5239303 Variance is: 2.778557
Iter: 13 The mean Y0 is 2.7748768 Variance is: 3.0619826
Iter: 14 The mean Y0 is 2.5047007 Variance is: 3.1440282
Iter: 15 The mean Y0 is 2.5259528 Variance is: 2.9251037
Iter: 16 The mean Y0 is 2.6737058 Variance is: 3.1377964
Iter: 17 The mean Y0 is 2.5601134 Variance is: 3.1039

In [18]:
df=pd.DataFrame()
df['y_pred']=y0_mean
df['loss_var']=loss_vec

df_temp=df[41:101]
lw=df_temp.loss_var.mean()-2.0*df_temp.loss_var.std()
up=df_temp.loss_var.mean()+2.0*df_temp.loss_var.std()
df1=df_temp[df_temp.loss_var>=lw]
df1=df1[df1.loss_var<=up]
df1.y_pred.mean()

2.60259104583223

In [25]:
import time

In [26]:
for epoch in np.arange(32,51,1):
    model_PPDE_bw=Lookback_PPDE_Backward(cfg)
    optimizer=optim.Adam(model_PPDE_bw.parameters(),lr=5e-4)
    grad_clip=0.1
    y0_mean=[];
    loss_vec=[];
    
    start_time = time.time()

    for i in range(110):
        batch_sig, batch_dw, batch_y,batch_x, batch_sel =generate_samples(model_PPDE_bw,batch_in=500)

        x_temp=model_PPDE_bw(batch_sig,batch_dw,batch_y,batch_x, batch_sel)
        loss_temp=loss_var(x_temp)

        optimizer.zero_grad()
        loss_temp.backward()
        optimizer.step()
    #   scheduler.step()
        y0_val=x_temp.mean().cpu().detach().numpy()
        loss_val=loss_temp.cpu().detach().numpy()

        y0_mean.append(y0_val)
        loss_vec.append(loss_val)
        if grad_clip: 
            nn.utils.clip_grad_value_(model_PPDE_bw.parameters(), grad_clip)

        print("Iter:", i, 'The mean Y0 is', y0_val , 'Variance is:' ,loss_val)
    
    print("Epoch:", epoch)
    
    print("--- %s seconds ---" % (time.time() - start_time))
    
    y_pred=np.array(y0_mean)
    loss_val=np.array(loss_vec)
    iters=np.arange(0,len(y_pred),1)+1
    df=pd.DataFrame()
    df['iter']=iters
    df['y_pred']=y_pred
    df['loss_var']=loss_val
    df.to_csv('eg2_trained_dataDim20/method2'+'_'+str(epoch)+'.csv')

Iter: 0 The mean Y0 is 2.609652 Variance is: 3.1237502
Iter: 1 The mean Y0 is 2.5960796 Variance is: 3.0634515
Iter: 2 The mean Y0 is 2.663824 Variance is: 3.2461596
Iter: 3 The mean Y0 is 2.656738 Variance is: 3.3336737
Iter: 4 The mean Y0 is 2.5575235 Variance is: 2.9359663
Iter: 5 The mean Y0 is 2.6721613 Variance is: 3.0496733
Iter: 6 The mean Y0 is 2.5052686 Variance is: 2.9479904
Iter: 7 The mean Y0 is 2.5510821 Variance is: 3.175502
Iter: 8 The mean Y0 is 2.6411903 Variance is: 2.9463916
Iter: 9 The mean Y0 is 2.5854466 Variance is: 2.9435184
Iter: 10 The mean Y0 is 2.6823955 Variance is: 3.406875
Iter: 11 The mean Y0 is 2.5253448 Variance is: 3.1265967
Iter: 12 The mean Y0 is 2.3911405 Variance is: 3.0386465
Iter: 13 The mean Y0 is 2.5397172 Variance is: 3.1242192
Iter: 14 The mean Y0 is 2.6679058 Variance is: 2.9509962
Iter: 15 The mean Y0 is 2.549501 Variance is: 3.101007
Iter: 16 The mean Y0 is 2.7257144 Variance is: 3.1798863
Iter: 17 The mean Y0 is 2.6073148 Variance is: 3

Iter: 35 The mean Y0 is 2.536001 Variance is: 2.6596985
Iter: 36 The mean Y0 is 2.6534111 Variance is: 3.3835518
Iter: 37 The mean Y0 is 2.599953 Variance is: 2.924873
Iter: 38 The mean Y0 is 2.645102 Variance is: 3.2344508
Iter: 39 The mean Y0 is 2.5676832 Variance is: 2.9950042
Iter: 40 The mean Y0 is 2.6754928 Variance is: 3.3115222
Iter: 41 The mean Y0 is 2.7498639 Variance is: 3.3887465
Iter: 42 The mean Y0 is 2.5667832 Variance is: 2.8160262
Iter: 43 The mean Y0 is 2.5030818 Variance is: 2.863455
Iter: 44 The mean Y0 is 2.5978754 Variance is: 3.1553342
Iter: 45 The mean Y0 is 2.752498 Variance is: 3.4540803
Iter: 46 The mean Y0 is 2.6435537 Variance is: 3.1414077
Iter: 47 The mean Y0 is 2.7294505 Variance is: 3.2984436
Iter: 48 The mean Y0 is 2.6493962 Variance is: 3.2036302
Iter: 49 The mean Y0 is 2.7090578 Variance is: 3.2939866
Iter: 50 The mean Y0 is 2.4694176 Variance is: 2.7340138
Iter: 51 The mean Y0 is 2.5661254 Variance is: 3.1010194
Iter: 52 The mean Y0 is 2.616572 Vari

Iter: 70 The mean Y0 is 2.5563607 Variance is: 3.076244
Iter: 71 The mean Y0 is 2.671173 Variance is: 3.2440114
Iter: 72 The mean Y0 is 2.635294 Variance is: 3.174472
Iter: 73 The mean Y0 is 2.714387 Variance is: 2.9796267
Iter: 74 The mean Y0 is 2.4568565 Variance is: 2.9548724
Iter: 75 The mean Y0 is 2.6261454 Variance is: 2.8982422
Iter: 76 The mean Y0 is 2.615016 Variance is: 2.8589556
Iter: 77 The mean Y0 is 2.7011812 Variance is: 3.0296533
Iter: 78 The mean Y0 is 2.562646 Variance is: 3.2798007
Iter: 79 The mean Y0 is 2.5874493 Variance is: 3.1506736
Iter: 80 The mean Y0 is 2.6566672 Variance is: 3.028094
Iter: 81 The mean Y0 is 2.730298 Variance is: 3.1228492
Iter: 82 The mean Y0 is 2.6271205 Variance is: 3.0612354
Iter: 83 The mean Y0 is 2.6829548 Variance is: 2.9683287
Iter: 84 The mean Y0 is 2.5560827 Variance is: 3.198346
Iter: 85 The mean Y0 is 2.5228114 Variance is: 3.1023622
Iter: 86 The mean Y0 is 2.4653845 Variance is: 2.9499993
Iter: 87 The mean Y0 is 2.6436303 Varianc

Iter: 105 The mean Y0 is 2.6004696 Variance is: 3.178392
Iter: 106 The mean Y0 is 2.6010854 Variance is: 3.198316
Iter: 107 The mean Y0 is 2.5691135 Variance is: 3.0605626
Iter: 108 The mean Y0 is 2.5254846 Variance is: 3.448568
Iter: 109 The mean Y0 is 2.613834 Variance is: 3.1702883
Epoch: 35
--- 164.9592638015747 seconds ---
Iter: 0 The mean Y0 is 2.6178818 Variance is: 2.9873486
Iter: 1 The mean Y0 is 2.6125774 Variance is: 3.1899931
Iter: 2 The mean Y0 is 2.7587495 Variance is: 3.3409746
Iter: 3 The mean Y0 is 2.6295633 Variance is: 2.8564694
Iter: 4 The mean Y0 is 2.5396354 Variance is: 3.0922983
Iter: 5 The mean Y0 is 2.544473 Variance is: 3.1230433
Iter: 6 The mean Y0 is 2.560095 Variance is: 3.3106115
Iter: 7 The mean Y0 is 2.6392817 Variance is: 2.7056315
Iter: 8 The mean Y0 is 2.613084 Variance is: 3.1746402
Iter: 9 The mean Y0 is 2.620919 Variance is: 3.090522
Iter: 10 The mean Y0 is 2.6568952 Variance is: 3.0245917
Iter: 11 The mean Y0 is 2.630209 Variance is: 3.0339727
It

Iter: 29 The mean Y0 is 2.5669363 Variance is: 2.9217796
Iter: 30 The mean Y0 is 2.548307 Variance is: 3.2819273
Iter: 31 The mean Y0 is 2.497197 Variance is: 2.966468
Iter: 32 The mean Y0 is 2.5545216 Variance is: 3.2191312
Iter: 33 The mean Y0 is 2.5927844 Variance is: 3.0973053
Iter: 34 The mean Y0 is 2.5201292 Variance is: 2.9868007
Iter: 35 The mean Y0 is 2.5097313 Variance is: 3.116405
Iter: 36 The mean Y0 is 2.6731985 Variance is: 2.9953651
Iter: 37 The mean Y0 is 2.5870848 Variance is: 3.091622
Iter: 38 The mean Y0 is 2.5139978 Variance is: 2.9978545
Iter: 39 The mean Y0 is 2.6420832 Variance is: 2.7418227
Iter: 40 The mean Y0 is 2.5551994 Variance is: 3.057008
Iter: 41 The mean Y0 is 2.6126983 Variance is: 2.9669666
Iter: 42 The mean Y0 is 2.6736765 Variance is: 2.9587119
Iter: 43 The mean Y0 is 2.6692007 Variance is: 2.80099
Iter: 44 The mean Y0 is 2.68172 Variance is: 3.0604792
Iter: 45 The mean Y0 is 2.5411038 Variance is: 3.1221642
Iter: 46 The mean Y0 is 2.6010833 Varianc

...

Iter: 17 The mean Y0 is 2.5316727 Variance is: 2.9304595
Iter: 18 The mean Y0 is 2.7733986 Variance is: 3.3596687
Iter: 19 The mean Y0 is 2.5435927 Variance is: 2.9591832
Iter: 20 The mean Y0 is 2.705503 Variance is: 3.2915723
Iter: 21 The mean Y0 is 2.5762026 Variance is: 3.1627064
Iter: 22 The mean Y0 is 2.731091 Variance is: 3.1023545
Iter: 23 The mean Y0 is 2.4615815 Variance is: 3.1654785
Iter: 24 The mean Y0 is 2.63295 Variance is: 2.8500175
Iter: 25 The mean Y0 is 2.536191 Variance is: 2.921155
Iter: 26 The mean Y0 is 2.4965565 Variance is: 3.0155165
Iter: 27 The mean Y0 is 2.5192356 Variance is: 3.017737
Iter: 28 The mean Y0 is 2.5183103 Variance is: 3.2575123
Iter: 29 The mean Y0 is 2.5429654 Variance is: 3.1623688
Iter: 30 The mean Y0 is 2.5480242 Variance is: 2.979718
Iter: 31 The mean Y0 is 2.446487 Variance is: 2.884071
Iter: 32 The mean Y0 is 2.6350095 Variance is: 2.9850178
Iter: 33 The mean Y0 is 2.6350055 Variance is: 3.1254487
Iter: 34 The mean Y0 is 2.7023356 Varianc

Iter: 52 The mean Y0 is 2.5372846 Variance is: 2.9634697
Iter: 53 The mean Y0 is 2.615846 Variance is: 3.0533812
Iter: 54 The mean Y0 is 2.5815718 Variance is: 3.4200609
Iter: 55 The mean Y0 is 2.6037736 Variance is: 3.0126312
Iter: 56 The mean Y0 is 2.6516285 Variance is: 3.0478451
Iter: 57 The mean Y0 is 2.5910554 Variance is: 2.8087878
Iter: 58 The mean Y0 is 2.5205467 Variance is: 3.0680149
Iter: 59 The mean Y0 is 2.679933 Variance is: 3.0744221
Iter: 60 The mean Y0 is 2.7118754 Variance is: 3.3161588
Iter: 61 The mean Y0 is 2.7366464 Variance is: 3.3935218
Iter: 62 The mean Y0 is 2.7188835 Variance is: 3.2572124
Iter: 63 The mean Y0 is 2.6681983 Variance is: 3.022402
Iter: 64 The mean Y0 is 2.7263486 Variance is: 3.2091126
Iter: 65 The mean Y0 is 2.512184 Variance is: 3.0814085
Iter: 66 The mean Y0 is 2.5720978 Variance is: 3.1718934
Iter: 67 The mean Y0 is 2.5784168 Variance is: 3.07886
Iter: 68 The mean Y0 is 2.6667197 Variance is: 2.9631064
Iter: 69 The mean Y0 is 2.7429173 Var

Iter: 87 The mean Y0 is 2.4956586 Variance is: 3.0117812
Iter: 88 The mean Y0 is 2.6550388 Variance is: 3.0251324
Iter: 89 The mean Y0 is 2.631628 Variance is: 2.8954978
Iter: 90 The mean Y0 is 2.5691035 Variance is: 2.8661482
Iter: 91 The mean Y0 is 2.7003224 Variance is: 3.1007717
Iter: 92 The mean Y0 is 2.6520433 Variance is: 3.138067
Iter: 93 The mean Y0 is 2.6804612 Variance is: 3.1563003
Iter: 94 The mean Y0 is 2.5849679 Variance is: 2.983409
Iter: 95 The mean Y0 is 2.737916 Variance is: 3.0945692
Iter: 96 The mean Y0 is 2.639 Variance is: 2.9100912
Iter: 97 The mean Y0 is 2.5443878 Variance is: 2.9628797
Iter: 98 The mean Y0 is 2.4790878 Variance is: 3.1768396
Iter: 99 The mean Y0 is 2.611816 Variance is: 3.267243
Iter: 100 The mean Y0 is 2.7006383 Variance is: 2.9474545
Iter: 101 The mean Y0 is 2.5392444 Variance is: 3.1715963
Iter: 102 The mean Y0 is 2.6507986 Variance is: 3.133645
Iter: 103 The mean Y0 is 2.5170057 Variance is: 3.1584861
Iter: 104 The mean Y0 is 2.5889587 Var

Iter: 11 The mean Y0 is 2.534881 Variance is: 3.0613215
Iter: 12 The mean Y0 is 2.5343828 Variance is: 2.9567356
Iter: 13 The mean Y0 is 2.6354764 Variance is: 2.9714975
Iter: 14 The mean Y0 is 2.5705729 Variance is: 3.0529065
Iter: 15 The mean Y0 is 2.6375964 Variance is: 3.480206
Iter: 16 The mean Y0 is 2.5842643 Variance is: 3.2644153
Iter: 17 The mean Y0 is 2.6413643 Variance is: 3.3259544
Iter: 18 The mean Y0 is 2.5681994 Variance is: 2.8100555
Iter: 19 The mean Y0 is 2.5048358 Variance is: 2.9550357
Iter: 20 The mean Y0 is 2.6338153 Variance is: 3.4087372
Iter: 21 The mean Y0 is 2.4666402 Variance is: 3.21413
Iter: 22 The mean Y0 is 2.5690968 Variance is: 2.8616054
Iter: 23 The mean Y0 is 2.5898035 Variance is: 3.1956384
Iter: 24 The mean Y0 is 2.6451259 Variance is: 3.06501
Iter: 25 The mean Y0 is 2.6287665 Variance is: 3.4084966
Iter: 26 The mean Y0 is 2.5570257 Variance is: 3.2559533
Iter: 27 The mean Y0 is 2.5680559 Variance is: 3.0133536
Iter: 28 The mean Y0 is 2.613258 Vari

Iter: 46 The mean Y0 is 2.58041 Variance is: 3.0991359
Iter: 47 The mean Y0 is 2.4791615 Variance is: 2.9351423
Iter: 48 The mean Y0 is 2.5976024 Variance is: 3.2133946
Iter: 49 The mean Y0 is 2.6040392 Variance is: 3.20203
Iter: 50 The mean Y0 is 2.6410038 Variance is: 2.9702811
Iter: 51 The mean Y0 is 2.6482472 Variance is: 3.3584895
Iter: 52 The mean Y0 is 2.617947 Variance is: 3.272175
Iter: 53 The mean Y0 is 2.5967245 Variance is: 2.8939397
Iter: 54 The mean Y0 is 2.5465443 Variance is: 2.8439252
Iter: 55 The mean Y0 is 2.5835502 Variance is: 2.9830694
Iter: 56 The mean Y0 is 2.5630653 Variance is: 3.0025277
Iter: 57 The mean Y0 is 2.5794423 Variance is: 2.8969276
Iter: 58 The mean Y0 is 2.7037973 Variance is: 2.9302037
Iter: 59 The mean Y0 is 2.6618404 Variance is: 3.013527
Iter: 60 The mean Y0 is 2.707902 Variance is: 2.9752173
Iter: 61 The mean Y0 is 2.6078048 Variance is: 3.1992235
Iter: 62 The mean Y0 is 2.710318 Variance is: 3.4288669
Iter: 63 The mean Y0 is 2.5748818 Varian

In [27]:
import os 
files_dir=os.getcwd()+'\\eg2_trained_dataDim20\\'
file_names=os.listdir(files_dir)
pred_vec=[]

In [28]:
for file in file_names:
    df=pd.read_csv(files_dir+file)
    df_temp=df[41:]
    lw=df_temp.loss_var.mean()-2.0*df_temp.loss_var.std()
    up=df_temp.loss_var.mean()+2.0*df_temp.loss_var.std()
    df1=df_temp[df_temp.loss_var>=lw]
    df1=df1[df1.loss_var<=up]
    
    pred_vec.append(df1.y_pred.mean())

In [29]:
est_vec=np.array(pred_vec)

In [30]:
est_vec.mean()

2.607437195162818

In [31]:
2*est_vec.std()+ est_vec.mean()

2.6271113540182283

In [32]:
-2*est_vec.std()+ est_vec.mean()

2.5877630363074076